In [ ]:
%matplotlib inline

In [ ]:
from IPython.parallel import Client
rc = Client()
dv = rc[:]

In [ ]:
%%px --local
import sys
sys.path.append('/Users/josh/Documents/Code/pytpc')

In [ ]:
%%px --local
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytpc
from pytpc.constants import *
import seaborn as sns

In [ ]:
from scipy.stats import zscore
from matplotlib.colors import LogNorm

In [ ]:
%%px --local
from scipy.interpolate import UnivariateSpline
from scipy.integrate import quad

In [ ]:
sns.set_style('white')

In [ ]:
%%px --local
tilt = 7*degrees
clock = 6.25
shape = 332
vd_mag = -2.8

emag = 15e3
bmag = 0.5
ef, bf = pytpc.utilities.create_fields(emag, bmag, tilt)

vd = pytpc.simulation.drift_velocity_vector(vd_mag, emag, bmag, tilt)

pads = pytpc.generate_pad_plane()

In [ ]:
%%px --local
def do_simulate(pmass, pchg, pen, pazi, ppol):
    proj = pytpc.Particle(pmass, pchg, pen, position=(0, 0, 1.0), azimuth=pazi, polar=ppol)
    gas = pytpc.gases.GenericGas(4, 600., 2, 41.8)

    simres = pytpc.track(proj, gas, ef, bf)

    evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values * 1e3, simres['de'].values, clock, vd, 40., 
                                   proj.mass_num, shape, offset=100)
    
    return evt

In [ ]:
%%px --local
def run_event(pvec):
    try:
        evt = do_simulate(pvec.mass_in, pvec.chg_in, pvec.en_in, pvec.azi_in, pvec.pol_in)
        tot_en = evt.traces['data'].sum()
        fpts = pd.DataFrame(evt.xyzs(vd, clock, pads, peaks_only=True), 
                            columns=('x', 'y', 'z', 'a')).sort('z', ascending=True)
        splx = UnivariateSpline(fpts.z.values, fpts.x.values, s=len(fpts)*3)
        sply = UnivariateSpline(fpts.z.values, fpts.y.values, s=len(fpts)*3)
        dsplx = splx.derivative()
        dsply = sply.derivative()
        xmin, xmax = fpts.z.min(), fpts.z.max()
        trlen = quad(lambda z: np.sqrt(dsplx(z)**2 + dsply(z)**2 + 1), xmin, xmax)[0]
        
    except Exception as e:
        print(type(e), e)
        return np.nan, np.nan
    else:
        return trlen, tot_en

# Parameter generation

In [ ]:
ptypes = {'1H': {'mass': 1, 'charge': 1},
          '2H': {'mass': 2, 'charge': 1},
          '3H': {'mass': 3, 'charge': 1},
          '3He': {'mass': 3, 'charge': 2},
          '4He': {'mass': 4, 'charge': 2},
          '4Li': {'mass': 4, 'charge': 3},
          '5Li': {'mass': 5, 'charge': 3},
          '6Li': {'mass': 6, 'charge': 3},
          '6Be': {'mass': 6, 'charge': 4},
          '7Be': {'mass': 7, 'charge': 4},
          '8B' : {'mass': 8, 'charge': 5},
          '10B': {'mass': 10, 'charge': 5},
          '12C': {'mass': 12, 'charge': 6},
          '14C': {'mass': 14, 'charge': 6},
          '14N': {'mass': 14, 'charge': 7},
          '15N': {'mass': 15, 'charge': 7},
          '15O': {'mass': 15, 'charge': 8},
          '16O': {'mass': 16,  'charge': 8}}

In [ ]:
neach = 100
ntypes = len(ptypes)
nevts = neach * ntypes
params = pd.DataFrame(columns=('name', 'mass_in', 'chg_in', 'en_in', 'azi_in', 'pol_in'), index=range(nevts))
for i, (n, p) in enumerate(ptypes.items()):
    print(i, n, p)
    params.mass_in.iloc[i*neach:(i+1)*neach] = np.full(neach, p['mass'])
    params.chg_in.iloc[i*neach:(i+1)*neach] = np.full(neach, p['charge'])
    params.name.iloc[i*neach:(i+1)*neach] = [n] * neach
params.en_in = np.random.normal(2, 0.1, nevts)
params.azi_in = np.random.uniform(0, 2*pi, nevts)
params.pol_in = np.random.uniform(pi-pi/4, pi, nevts)

---

# Running

In [ ]:
rawres = dv.map_sync(lambda x: run_event(x[1]), params.iterrows())
piddat = pd.concat((params, pd.DataFrame(rawres, columns=('tracklen', 'total_en'))), axis=1)
piddat['mean_dedx'] = piddat.total_en / piddat.tracklen
piddat.to_pickle('piddat.p')

In [ ]:
piddat = pd.read_pickle('piddat.p')

In [ ]:
piddat = piddat.dropna()

In [ ]:
piddat = piddat[zscore(piddat.tracklen) < 3]

In [ ]:
pal = sns.choose_colorbrewer_palette('qualitative')

In [ ]:
piddat['ZoA_ratio'] = piddat.chg_in / piddat.mass_in

In [ ]:
piddat['entot_in'] = piddat.en_in * piddat.mass_in

In [ ]:
piddat['sqrten'] = piddat.total_en

In [ ]:
g = sns.FacetGrid(piddat, hue="name", size=8)
g.map(plt.scatter, 'mean_dedx', 'total_en', s=50, alpha=0.7, linewidth=.5, edgecolor="white")
g.add_legend();

In [ ]:
plt.hexbin(piddat.total_en, piddat.tracklen, cmap='BuGn')

---

In [ ]:
evt = do_simulate(*params.iloc[2][1:])

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
entot = evt.traces['data'].sum(); entot

In [ ]:
pts = pd.DataFrame(evt.xyzs(vd, clock, pads), columns=('x', 'y', 'z', 'a')).sort('z')
fpts = pd.DataFrame(evt.xyzs(vd, clock, pads, peaks_only=True), 
                    columns=('x', 'y', 'z', 'a')).sort('z', ascending=True)

In [ ]:
plt.scatter(fpts.z, fpts.x, c=fpts.a, cmap='BuGn', edgecolor='none', alpha=0.6)
plt.scatter(fpts.z, fpts.y, c=fpts.a, cmap='OrRd', edgecolor='none', alpha=0.6)
# plt.gca().set_aspect('equal')

In [ ]:
plt.plot(pts.z)

In [ ]:
splx = UnivariateSpline(fpts.z.values, fpts.x.values, s=len(fpts.z.values)*3, ext=0)
sply = UnivariateSpline(fpts.z.values, fpts.y.values, s=len(fpts.z.values)*3)
dsplx = splx.derivative()
dsply = sply.derivative()

In [ ]:
plt.scatter(pts.z, pts.x, c=pts.a, cmap='BuGn', edgecolor='none', alpha=0.6)
plt.scatter(pts.z, pts.y, c=pts.a, cmap='OrRd', edgecolor='none', alpha=0.6)
plt.ylim(-275,275)
xmin, xmax = pts.z.min(), pts.z.max()
xvals = np.linspace(xmin, xmax, 100)
plt.plot(xvals, splx(xvals), color='Teal')
plt.plot(xvals, sply(xvals), color='Orange')

In [ ]:
splx(xvals)